In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mahmoud-dataset/README
/kaggle/input/mahmoud-dataset/dataset_description.json
/kaggle/input/mahmoud-dataset/participants.json
/kaggle/input/mahmoud-dataset/participants.tsv
/kaggle/input/mahmoud-dataset/CHANGES
/kaggle/input/mahmoud-dataset/sub-654178/sub-654178_scans.tsv
/kaggle/input/mahmoud-dataset/sub-654178/sub-654178_scans.json
/kaggle/input/mahmoud-dataset/sub-654178/anat/sub-654178_acq-headmotion1_T1w.json
/kaggle/input/mahmoud-dataset/sub-654178/anat/sub-654178_acq-standard_T1w.json
/kaggle/input/mahmoud-dataset/sub-654178/anat/sub-654178_acq-headmotion2_T1w.json
/kaggle/input/mahmoud-dataset/sub-654178/anat/sub-654178_acq-headmotion1_T1w.nii/sub-654178_ses-1_acq-move1_T1w.nii
/kaggle/input/mahmoud-dataset/sub-654178/anat/sub-654178_acq-headmotion2_T1w.nii/sub-654178_ses-1_acq-move2_T1w.nii
/kaggle/input/mahmoud-dataset/sub-654178/anat/sub-654178_acq-standard_T1w.nii/sub-654178_ses-1_acq-conve_T1w.nii
/kaggle/input/mahmoud-dataset/sub-228373/sub-228373_scans.tsv


In [45]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

from matplotlib import pyplot as plt
from IPython import display

In [46]:
import os
import subprocess
import nibabel as nib
import tensorflow as tf
from skimage import exposure
from math import ceil
import matplotlib.pyplot as plt
import random

class DataLoader:
    def __init__(self, data_path, split_ratio, batch_size=None):
        self.data_path = data_path
        self.split_ratio = split_ratio
        self.all_subjects = None
        self.subjects_lists = []
        self.labels = {'train': 0, 'test': 1, 'validation': 2}
        self.size = [0, 0, 0]
        self.batch_size = batch_size
        self.slices_number = None

    def list_subjects(self):
        subjects = os.listdir(self.data_path)
        subjects = [item for item in subjects if item.startswith('sub')]
        subjects.reverse()
        self.all_subjects = subjects

    def get_nifti_path(self, subject, number_of_motion='1'):
        ref_path_stand = f'{self.data_path}/{subject}/anat/{subject}_acq-standard_T1w.nii/'
        select_path_stand = subprocess.run(['ls', ref_path_stand], capture_output=True, text=True).stdout.replace("\n", "")

        ref_path_motion = f'{self.data_path}/{subject}/anat/{subject}_acq-headmotion{number_of_motion}_T1w.nii/'
        select_path_motion = subprocess.run(['ls', ref_path_motion], capture_output=True, text=True).stdout.replace("\n", "")

        return [ref_path_stand + select_path_stand, ref_path_motion + select_path_motion]

    def get_paired_volumes(self, path):
        if os.path.exists(path[0]) and os.path.exists(path[1]):
            free_data = nib.load(path[0]).get_fdata()
#             free_data = exposure.rescale_intensity(free_data, out_range=(0.0, 1.0))
            free_data = exposure.rescale_intensity(free_data[37:-37], out_range=(-1.0, 1.0))

            motion_data = nib.load(path[1]).get_fdata()
#             motion_data = exposure.rescale_intensity(motion_data, out_range=(0.0, 1.0))
            motion_data = exposure.rescale_intensity(motion_data[37:-37], out_range=(-1.0, 1.0))
            return tf.convert_to_tensor(free_data), tf.convert_to_tensor(motion_data)
        else:
            return None, None

    def split_data(self):
        self.list_subjects()
        if ceil(sum(self.split_ratio)) == 1 and len(self.split_ratio) <= 3:
            self.split_ratio.insert(0, 0)
            cumulative_sum = [sum(self.split_ratio[:i + 1]) for i in range(len(self.split_ratio))]
            number_of_subjects = len(self.all_subjects)

            for i in range(1, len(self.split_ratio)):
                self.subjects_lists.append(
                    self.all_subjects[int(round(cumulative_sum[i - 1] * number_of_subjects)):int(
                        round(cumulative_sum[i] * number_of_subjects))])

                self.size[i - 1] = len(self.subjects_lists[i - 1])  * 2 * 190

                if i - 1 == 0:
                    self.size[i - 1] -= 8  * 2 * 190
        else:
            print("The Summation of ratios is not equal to 1")
       
    def generator(self, mode):
        subjects = self.subjects_lists[self.labels[mode]]

        def data_gen():
            for subject in subjects:
                for i in range(2):
                    pathes = self.get_nifti_path(subject, str(i + 1))
                    free, motion = self.get_paired_volumes(pathes)
                    if motion is not None:
                        self.slices_number = motion.shape[0]

                        for slice_id in range(0, self.slices_number):
                            start_idx = slice_id + 1
                            end_idx = (slice_id + 1) + 1
                            if (end_idx < self.slices_number-1):
                                free_slice = free[start_idx:end_idx]
                                free_slice = tf.transpose(free_slice, perm=[1, 2, 0])
                                
                                motion_slice = motion[start_idx:end_idx]
                                motion_slice = tf.transpose(motion_slice, perm=[1, 2, 0])
                                
#                                 motion_before_slice = motion[start_idx-1:end_idx-1]
#                                 motion_before_slice = tf.transpose(motion_before_slice, perm=[1, 2, 0])
                                
#                                 motion_after_slice = motion[start_idx+1:end_idx+1]
#                                 motion_after_slice = tf.transpose(motion_after_slice, perm=[1, 2, 0])

                                yield (
                                motion_slice,
                                free_slice
                                )

        input_signature = (tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32)
        )

        dataset = tf.data.Dataset.from_generator(data_gen, output_signature=input_signature)
        dataset = dataset.batch(self.batch_size)

        return dataset

In [47]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
import math
import pandas as pd
from tensorflow.keras.models import model_from_json

# Constants
TRAIN = 1  # True False
TEST = 0  # True False
# NB_EPOCH = 20
# LEARNING_RATE = 0.001  # 0.001 (default)
# HEIGHT, WIDTH = 256, 256
PREDICTION_PATH = '/kaggle/working/Prediction'
WEIGHTS_PATH = '/kaggle/working/Weights'

print('Reading Data ... ')
data_path = "/kaggle/input/mahmoud-dataset"
split_ratio = [0.7, 0.2, 0.1]
# split_ratio = [0.03, 0.92, 0.03]
# batch_size = 14
BATCH_SIZE = 1
EPOCHS = 10
data_loader = DataLoader(data_path, split_ratio, BATCH_SIZE)
data_loader.split_data()

train_dataset = data_loader.generator('train')
test_dataset = data_loader.generator('test')
validation_dataset = data_loader.generator('validation')

Reading Data ... 


In [ ]:
# The facade training set consist of 400 images
BUFFER_SIZE = 400
# The batch size of 1 produced better results for the U-Net in the original pix2pix experiment
LAMBDA = 100
# Each image is 256x256 in size
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

In [ ]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [ ]:
def Generator():
  inputs = tf.keras.layers.Input(shape=[256, 256, 1])

  down_stack = [
    downsample(64, 4, apply_batchnorm=False),  # (batch_size, 128, 128, 64)
    downsample(128, 4),  # (batch_size, 64, 64, 128)
    downsample(256, 4),  # (batch_size, 32, 32, 256)
    downsample(512, 4),  # (batch_size, 16, 16, 512)
    downsample(512, 4),  # (batch_size, 8, 8, 512)
    downsample(512, 4),  # (batch_size, 4, 4, 512)
    downsample(512, 4),  # (batch_size, 2, 2, 512)
    downsample(512, 4),  # (batch_size, 1, 1, 512)
  ]

  up_stack = [
    upsample(512, 4, apply_dropout=True),  # (batch_size, 2, 2, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 4, 4, 1024)
    upsample(512, 4, apply_dropout=True),  # (batch_size, 8, 8, 1024)
    upsample(512, 4),  # (batch_size, 16, 16, 1024)
    upsample(256, 4),  # (batch_size, 32, 32, 512)
    upsample(128, 4),  # (batch_size, 64, 64, 256)
    upsample(64, 4),  # (batch_size, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(1, 4,
                                         strides=2,
                                         padding='same',
                                         kernel_initializer=initializer,
                                         activation='tanh')  # (batch_size, 256, 256, 3)

  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = skips[:-1]
  skips.reverse()
  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = tf.keras.layers.Concatenate()([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
generator = Generator()

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # Mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

In [ ]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[256, 256, 1], name='input_image')
  tar = tf.keras.layers.Input(shape=[256, 256, 1], name='target_image')

  x = tf.keras.layers.concatenate([inp, tar])  # (batch_size, 256, 256, channels*2)

  down1 = downsample(64, 4, False)(x)  # (batch_size, 128, 128, 64)
  down2 = downsample(128, 4)(down1)  # (batch_size, 64, 64, 128)
  down3 = downsample(256, 4)(down2)  # (batch_size, 32, 32, 256)

  zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (batch_size, 34, 34, 256)
  conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1)  # (batch_size, 31, 31, 512)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (batch_size, 33, 33, 512)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2)  # (batch_size, 30, 30, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)

In [ ]:
discriminator = Discriminator()

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = '/kaggle/working/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
@tf.function
def train_step(input_image, target):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator([input_image, target], training=True)
    disc_generated_output = discriminator([input_image, gen_output], training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))
#   with summary_writer.as_default():
#     tf.summary.scalar('gen_total_loss', gen_total_loss, step=step//1000)
#     tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=step//1000)
#     tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=step//1000)
#     tf.summary.scalar('disc_loss', disc_loss, step=step//1000)

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5,cmap='gray')
    plt.axis('off')
  plt.show()

In [ ]:
import csv

def ssim_score(y_true, y_pred):
    score = tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0, filter_size=5, filter_sigma=1.5))
    return score
def mse_score(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    return mse
def psnr(y_true, y_pred):
    return tf.reduce_mean(-tf.image.psnr(y_true, y_pred, max_val=2.0))
def fit(train_ds, epochs):
    with open('metrics.csv', mode='w', newline='') as metrics_file:
        metrics_writer = csv.writer(metrics_file)
        metrics_writer.writerow(['Epoch', 'SSIM', 'PSNR', 'MSE'])
        
        for epoch in range(epochs):
            step = 0
            ssim_values = []
            psnr_values = []
            mse_values = []

            for motion, free in train_ds:
                train_step(motion, free)
                step += 1
                prediction = generator(motion, training=False)
                mse_values.append(mse_score(free, prediction))
                ssim_values.append(ssim_score(free, prediction))
                psnr_values.append(psnr(free, prediction))

            # Calculate mean of metrics
            avg_ssim = tf.reduce_mean(ssim_values)
            avg_psnr = tf.reduce_mean(psnr_values)
            avg_mse = tf.reduce_mean(mse_values)

            # Print or log metrics for the epoch
            print(f"Epoch {epoch + 1}, SSIM: {avg_ssim}, PSNR: {avg_psnr}, MSE: {avg_mse}")

            # Save metrics to CSV
            metrics_writer.writerow([epoch + 1, avg_ssim.numpy(), avg_psnr.numpy(), avg_mse.numpy()])

            # Save model
            generator.save(f"/kaggle/working/pix2pix_model{epoch}.h5")


In [ ]:
fit(train_dataset,10)

In [ ]:
import torch
from skimage.metrics import structural_similarity as ssim
from scipy import ndimage
from tensorflow.keras.models import load_model
angle = 90
# Load the model
# model = torch.load('/kaggle/input/unit-3-epochs/gen_00073632.pt')
# testing = UNIT_Trainer(config)
# # If you're loading the model on a GPU and want to move it to GPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# gen_a = model['a']
# gen_b = model['b']
# # disc_a.input_shape
# # disc_a.to(device)
# testing.gen_a.load_state_dict(gen_a)
# testing.gen_b.load_state_dict(gen_b)
# testing.cuda()
# testing.eval()
# t2.gen_a.load_state_dict(gen_b)
# t2.cuda()
# t2.eval()
# T_E = t.gen_a.encode


# Load the model
loaded_model = load_model("/kaggle/working/pix2pixmodel9.h5")

# Define a checkpoint manager


no_slice = 37
motion_vol = nib.load('/kaggle/input/mahmoud-dataset/sub-122916/anat/sub-122916_acq-headmotion2_T1w.nii/sub-122916_ses-1_acq-move2_T1w.nii').get_fdata()
free_vol = nib.load('/kaggle/input/mahmoud-dataset/sub-122916/anat/sub-122916_acq-standard_T1w.nii/sub-122916_ses-1_acq-conve_T1w.nii').get_fdata()

motion_vol = exposure.rescale_intensity(motion_vol[37:-37], out_range=(-1.0, 1.0))
free_vol = exposure.rescale_intensity(free_vol[37:-37], out_range=(-1.0, 1.0))
# data = torch.tensor(motion_vol[no_slice].reshape(1, 1, 256, 256), device=device).float()

# loaded_model.summary()
out_np = loaded_model.predict(motion_vol[no_slice].reshape(1,256,256,1))
# out = testing.gen_b.decode(o1[0])
# out_np = out.detach().cpu().numpy()
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
             # Plot images on each subplot
rotated_motion = ndimage.rotate(motion_vol[no_slice], angle)

axs[0].imshow(rotated_motion.reshape(256,256,1),cmap='gray')
axs[0].axis('off')  # Hide axis
axs[0].set_title('Motion Image')
rotated_free = ndimage.rotate(free_vol[no_slice], angle)
axs[1].imshow(rotated_free.reshape(256,256,1),cmap='gray')
axs[1].axis('off')  # Hide axis
axs[1].set_title('Free Image')
rotated_model = ndimage.rotate(out_np.reshape(256,256,1), angle)
axs[2].imshow(rotated_model.reshape(256,256,1),cmap='gray')
axs[2].axis('off')  # Hide axis
axs[2].set_title('Model Image')

# free_slice = free_vol[no_slice].reshape(256,256,1)
# out = out_np.reshape(256,256,1)
# ssim_score = ssim(free_slice,out, win_size=5 ,multichannel=False,data_range=2.0)
#             ssim_score_tensor = torch.tensor(ssim_score, dtype=torch.float32)
# print("SSIM score:", ssim_score)
# Adjust layout
plt.tight_layout()

plt.show()
# plt.imshow(out_np.reshape(256,256,1),cmap='gray')
# Now your model is ready to use
